In [2]:
import math
import numpy as np
import networkx as nx
import minorminer

import plotly.graph_objects as go
import matplotlib.pyplot as plt 
import matplotlib.patches as patches

# !pip install -e .
import os
os.chdir("./src")
from nodal_knot import NodalKnot
os.chdir("..")

In [3]:
def k_to_zw(kx, ky, kz):
    """ F: 3D Brillouin zone -> C^2 """

    z_real = np.cos(2*kz) + 0.5
    z_imag = np.cos(kx) + np.cos(ky) + np.cos(kz) - 2.0
    z = z_real + 1j*z_imag
    
    w_real = np.sin(kx)
    w_imag = np.sin(ky)
    w = w_real + 1j*w_imag

    return z, w

def zw_to_c_hopf(z, w):
    """ f: C^2 -> C (Hopf Link) """
    return np.power(z, 2) - np.power(w, 2)

def zw_to_c_trefoil(z, w):
    """ f: C^2 -> C (Trefoil Knot) """
    return np.power(z, 2) - np.power(w, 3)

def zw_to_c_3link(z, w):
    """ f: C^2 -> C (Figure-8 Knot) """
    return np.power(z, 3) - np.power(w, 2)*z

hopf = NodalKnot(k_to_zw, zw_to_c_hopf)
trefoil = NodalKnot(k_to_zw, zw_to_c_trefoil)
threelink = NodalKnot(k_to_zw, zw_to_c_3link)

## Thickened Knot

In [ ]:
trefoil_surface = trefoil.knot_surface_points(thickness=0.2, epsilon=0.001)
trefoil_surf_fig = trefoil.plot_3D(trefoil_surface,
                            file_name="./assets/trefoil_tube.html"
                        )
trefoil_surf_fig.show()

# Knotted Graph

In [ ]:
def get_edge_pts(G):
    pts_list = []
    for u, v, data in G.edges(data=True):
        pts = data.get('pts')
        pts_list.append(pts)
    # Combine all edge pts into one array.
    if pts_list:
        all_pts = np.vstack(pts_list)
    return all_pts

trefoil_graph = trefoil.skeleton_graph(clean=True, thickness=0.2)
trefoil_skeleton = get_edge_pts(trefoil_graph)
trefoil_ske_fig = trefoil.plot_3D(trefoil_skeleton,
                        file_name="./assets/trefoil_skeleton.html"
                    )
trefoil_graph_fig = trefoil.plot_graph(trefoil_graph,
                        file_name="./assets/trefoil_knotted_graph.html"
                    )
trefoil_ske_fig.show()
trefoil_graph_fig.show()

# Intrinsically Linked - Petersen Graph Minor Example

In [ ]:
def arbitrary_func(z, w):
    """ f: C^2 -> C (Figure-8 Knot) """   
    return  z*(z**2-w**4+w)
 
k=0.9*np.pi
X = NodalKnot(k_to_zw, arbitrary_func,
              kx_min=-k, kx_max=k,
              ky_min=-k, ky_max=k,
              kz_min=0, kz_max=k,pts_per_dim=600)
X_surface = X.knot_surface_points(thickness=0.2, epsilon=0.001)


X_points = X.knot_skeleton_points()
X_fig = trefoil.plot_3D(X_points)
X_fig.show()
 
 

In [ ]:

X_surf_fig = X.plot_3D(X_surface)
X_surf_fig.show()
 

In [ ]:
X_graph = X.skeleton_graph(clean=True, thickness=0.2)
X_graph_fig = X.plot_graph(X_graph)
X_graph_fig.show()

In [ ]:
X.print_graph_properties(X_graph)

In [ ]:
import networkx as nx

def contract_degree_two_nodes(G):
    # Work on a copy if you wish to preserve the original graph.
    G = G.copy()
    # We use a loop that repeats until no degree-2 node remains.
    changed = True
    while changed:
        changed = False
        # We iterate over a static list of nodes because we will be modifying G.
        for node in list(G.nodes()):
            if G.degree(node) == 2:
                neighbors = list(G.neighbors(node))
                if len(neighbors) == 2:
                    u, w = neighbors
                    # Optionally, check for self-loops or parallel edges.
                    if u != w:
                        # Add an edge between the neighbors.
                        G.add_edge(u, w)
                    # Remove the degree-2 node.
                    G.remove_node(node)
                    changed = True
    return G 
simplified_graph = contract_degree_two_nodes(X_graph)
mapping = {old_label: new_label for new_label, old_label in enumerate(simplified_graph.nodes(), start=1)}
simplified_graph = nx.relabel_nodes(simplified_graph, mapping)

# Assuming contracted_graph is your graph after processing:
X.print_graph_properties(simplified_graph)


In [ ]:
# Generate positions using the shell layout.
pos = nx.shell_layout(simplified_graph)

plt.figure(figsize=(4, 4))
nx.draw(simplified_graph, pos, with_labels=True,
        node_color='white', edgecolors='black',  # white fill, black border for nodes
        edge_color='black', node_size=500) 
plt.show()


##### 1 - To do: Add whole petersen family of graphs to create a function Is_Intrinsically_Linked (Sage liblary has it)
 

In [ ]:
petersen_graph= nx.petersen_graph() ### Try to add whole petersen family of graphs to create a function Is_Intrinsically_Linked
Embedding=X.check_minor(host_graph=simplified_graph,minor_graph=petersen_graph)

In [ ]:
def standard_petersen_layout():
    """
    Returns a dictionary of positions for the Petersen graph in a classic layout:
      - Nodes 0..4 arranged in a pentagon (outer ring)
      - Nodes 5..9 arranged in a smaller, rotated pentagon (inner star)
    """
    coords = {}
    R_outer = 1.2
    R_inner = 0.7
    # Outer ring: nodes 0..4
    for i in range(5):
        angle = 2 * math.pi * i / 5
        x = R_outer * math.cos(angle)
        y = R_outer * math.sin(angle)
        coords[i] = (x, y)
    # Inner star: nodes 5..9, rotated half a step
    for i in range(5):
        angle = 2 * math.pi * (i + 0.5) / 5
        x = R_inner * math.cos(angle)
        y = R_inner * math.sin(angle)
        coords[i + 5] = (x, y)
    return coords
# Get canonical positions for the Petersen minor vertices.
petersen_positions = standard_petersen_layout()

# Create a new figure.
fig, ax = plt.subplots(figsize=(8, 8))
 
scale_factor = 1.0  # already scaled in standard_petersen_layout
for (u, v) in petersen_graph.edges():
    pos_u = np.array(petersen_positions[u]) * scale_factor
    pos_v = np.array(petersen_positions[v]) * scale_factor
    ax.plot([pos_u[0], pos_v[0]], [pos_u[1], pos_v[1]], 'k-', lw=3, zorder=0)

# Parameters for drawing boxes.
box_width = 0.5
box_height = 0.3

# For each minor vertex (0 through 9), create a box at the canonical position.
for minor_node, (x_center, y_center) in petersen_positions.items():
    # Calculate bottom-left corner of the box.
    x_box = x_center - box_width / 2
    y_box = y_center - box_height / 2
    
    # Set box fill color: outer (0–4) blue, inner (5–9) red.
    if minor_node < 5:
        box_color = 'lightblue'
    else:
        box_color = 'lightcoral'
    
    # Create a rectangle patch representing the group (chain) of nodes.
    rect = patches.Rectangle((x_box, y_box), box_width, box_height,
                                edgecolor='black', facecolor=box_color, lw=2, zorder=1)
    ax.add_patch(rect)
    
    # Get the chain (list of host nodes) for this minor vertex.
    chain = Embedding.get(minor_node, [])
    n = len(chain)
    if n == 0:
        continue
    
    # Draw each host node as a circle with its number inside.
    circle_radius = box_height / 2.5  # increased radius for bigger circles
    
    # Determine horizontal positions for the circles inside the box.
    if n == 1:
        x_positions = [x_center]
    else:
        x_positions = np.linspace(x_box + circle_radius, x_box + box_width - circle_radius, n)
    
    # Vertical position for all circles is the center of the box.
    y_pos = y_center
    
    # Draw each circle (with a high zorder so they appear on top of edges).
    for i, node_val in enumerate(chain):
        circ_center = (x_positions[i], y_pos)
        circ = patches.Circle(circ_center, radius=circle_radius,
                                edgecolor='black', facecolor='white', lw=2, zorder=10)
        ax.add_patch(circ)
        ax.text(circ_center[0], circ_center[1], str(node_val),
                ha='center', va='center', fontsize=10, fontweight='bold', zorder=11)
        
    
ax.set_aspect('equal')
ax.axis('off') 
plt.show()


##### 2 - To do: Try to find a intrinsically knotted(IK) example containing K6, K7 (remember IK is a minor closed property so if we can find an example where K6,K7 is a minor of a given G, then G is IK)

##### 3- To do: Do it for intrinsic n-linkedness for n>=3

# Intrinsically Knotted - $K_7$ & $K_{3,3,1,1}$  Conway Foisy  Theorem

In [ ]:
 
def arbitrary_func(z, w):
 
    return (z**2+w**2)+(z**3-w**4)
 
 
k=np.pi
X = NodalKnot(k_to_zw, arbitrary_func,
              kx_min=-k, kx_max=k,
              ky_min=-k, ky_max=k,
              kz_min=0, kz_max=k,pts_per_dim=400)
 

X_points = X.knot_skeleton_points()
X_fig = trefoil.plot_3D(X_points)
X_fig.show()
 
 

In [ ]:
X_graph = X.skeleton_graph(clean=True, thickness=0.6)
X_graph_fig = X.plot_graph(X_graph)
X_graph_fig.show()

In [ ]:
simplified_graph = contract_degree_two_nodes(X_graph)
mapping = {old_label: new_label for new_label, old_label in enumerate(simplified_graph.nodes(), start=1)}
simplified_graph = nx.relabel_nodes(simplified_graph, mapping)

# Assuming contracted_graph is your graph after processing:
X.print_graph_properties(simplified_graph)

In [ ]:
K7 = nx.complete_graph(7)
K3311 = nx.complete_multipartite_graph(3, 3, 1, 1)
X.check_minor(host_graph=simplified_graph,minor_graph=K7)
X.check_minor(host_graph=simplified_graph,minor_graph=K3311)

### Yamada Polynomial

In [ ]:
from topoly import yamada
from topoly.params import Closure

In [ ]:
def pd_code(graph): 
    # Create a dictionary to assign a unique id for each unordered edge pair
    edge_ids = {}
    current_id = 1
    
    # Iterate over all edges in the graph
    for u, v in graph.edges():
        key = tuple(sorted((u, v)))  # Ensure (u,v) and (v,u) are considered identical.
        if key not in edge_ids:
            edge_ids[key] = current_id
            current_id += 1

    # Create a dictionary mapping each node to its list of edge identifiers
    node_edge_ids_dict = {}
    for node in graph.nodes():
        node_edge_ids = []
        # For each neighbor of the node, get the corresponding edge id.
        for neighbor in graph[node]:
            key = tuple(sorted((node, neighbor)))
            node_edge_ids.append(edge_ids[key])
        node_edge_ids_dict[node] = node_edge_ids

    # Return only the list of edge identifier lists
    return list(node_edge_ids_dict.values())

X_graph = X.skeleton_graph(clean=True, thickness=0.6)
X_graph_fig = X.plot_graph(X_graph)
X_graph_fig.show()

simplified_graph = contract_degree_two_nodes(X_graph)
mapping = {old_label: new_label for new_label, old_label in enumerate(simplified_graph.nodes(), start=1)}
simplified_graph = nx.relabel_nodes(simplified_graph, mapping)

# Assuming contracted_graph is your graph after processing:
X.print_graph_properties(simplified_graph)


Pd_code = pd_code(simplified_graph)
 
print("Yamada polynomial coefficients are: " +yamada( list(Pd_code), closure=Closure.CLOSED))

In [ ]:
def arbitrary_func(z, w):
    return  z*(z**2-w**4+w)
 
k=0.9*np.pi
X = NodalKnot(k_to_zw, arbitrary_func,
              kx_min=-k, kx_max=k,
              ky_min=-k, ky_max=k,
              kz_min=0, kz_max=k,pts_per_dim=600)
X_graph = X.skeleton_graph(clean=True, thickness=0.6)
X_graph_fig = X.plot_graph(X_graph)
X_graph_fig.show()

simplified_graph = contract_degree_two_nodes(X_graph)
mapping = {old_label: new_label for new_label, old_label in enumerate(simplified_graph.nodes(), start=1)}
simplified_graph = nx.relabel_nodes(simplified_graph, mapping)

# Assuming contracted_graph is your graph after processing:
X.print_graph_properties(simplified_graph)


Pd_code = pd_code(simplified_graph)
 
print("Yamada polynomial coefficients are:" +yamada(Pd_code, closure=Closure.CLOSED))

In [ ]:
from yamada import extract_graph_from_json_file, SpatialGraph

nodes = list(map(str, simplified_graph.nodes()))

# Get node positions from the node attribute 'pos'
node_positions = nx.get_node_attributes(simplified_graph, 'pos') 

# Get the list of edges
edges = [(str(u), str(v)) for u, v in simplified_graph.edges()]


# Instantiate the SpatialGraph object
sg = SpatialGraph(nodes=nodes,
                  node_positions=node_positions,
                  edges=edges)

# Plot the Spatial Graph in 3D and the projected 2D plane to see what's going on. Crossings will be circled in red.
# Note: Crossings occur when two edges that do not intersect, but appear to when they are projected onto a 2D plane.
# sg.plot()

# Create the spatial graph diagram (necessary for calculating the Yamada polynomial)
sgd = sg.create_spatial_graph_diagram()

# Calculate the Yamada polynomial
# We use the normalized version because it is more useful for comparing polynomials
yamada_polynomial = sgd.normalized_yamada_polynomial()
print("Yamada Polynomial: ", yamada_polynomial)

In [ ]:
edges
